In [4]:
import pygame
import numpy as np
from scipy.stats import multivariate_normal

# --- 파티클 필터 클래스 정의 ---
class ParticleFilter:
    def __init__(self, num_particles, initial_state, initial_cov, Q, R, fx, hx):
        self.N = num_particles
        self.Q = Q
        self.R = R
        self.fx = fx
        self.hx = hx
        
        # 1. 초기화: 초기 상태 주변에 정규분포로 파티클 생성
        self.particles = np.random.multivariate_normal(initial_state, initial_cov, self.N)
        # 모든 파티클의 가중치는 동일하게 시작
        self.weights = np.full(self.N, 1.0 / self.N)
        
    def predict(self, u, dt):
        # 2. 예측: 모든 파티클을 상태 전이 함수에 따라 이동
        for i in range(self.N):
            self.particles[i] = self.fx(self.particles[i], u, dt)
        
        # 프로세스 노이즈 추가
        self.particles += np.random.multivariate_normal(np.zeros(len(Q)), Q, self.N)

    def update(self, z):
        # 3. 업데이트: 측정값 z를 기반으로 가중치 계산
        # 측정 노이즈가 가우시안이라고 가정하고 확률 밀도(likelihood) 계산
        likelihoods = np.array([multivariate_normal.pdf(z, mean=self.hx(p), cov=self.R) for p in self.particles])
        
        # 가중치 업데이트
        self.weights *= likelihoods
        # 가중치 정규화 (전체 합이 1이 되도록)
        weight_sum = np.sum(self.weights)
        if weight_sum < 1e-10:
             self.weights = np.full(self.N, 1.0 / self.N) # 모든 가중치가 0이 되면 리셋
        else:
             self.weights /= weight_sum

    def resample_if_needed(self):
        # 4. 리샘플링: 파티클 퇴화(degeneracy) 방지
        # 유효 파티클 수가 임계값(N/2) 이하로 떨어지면 리샘플링 수행
        N_eff = 1.0 / np.sum(self.weights**2)
        if N_eff < self.N / 2:
            indices = np.random.choice(np.arange(self.N), size=self.N, replace=True, p=self.weights)
            self.particles = self.particles[indices]
            self.weights.fill(1.0 / self.N) # 가중치 리셋

    def estimate(self):
        # 5. 최종 추정: 파티클들의 가중 평균 계산
        return np.average(self.particles, weights=self.weights, axis=0)

# --- 3D 시스템 모델 및 시각화 함수 (이전과 동일) ---
def state_transition_function(x, u, dt):
    phi, theta, psi = x; p, q, r = u
    epsilon = 1e-8; cos_theta = np.cos(theta)
    if abs(cos_theta) < epsilon: cos_theta = epsilon
    phi_dot = p + np.sin(phi)*np.tan(theta)*q + np.cos(phi)*np.tan(theta)*r
    theta_dot = np.cos(phi)*q - np.sin(phi)*r
    psi_dot = (np.sin(phi)/cos_theta)*q + (np.cos(phi)/cos_theta)*r
    return x + np.array([phi_dot, theta_dot, psi_dot]) * dt
def measurement_function(x): return np.array([x[0], x[1]])
def draw_cube(surface, angles_rad, center_pos, size, color, width):
    phi, theta, psi = angles_rad
    points = np.array([[-1,-1,-1],[1,-1,-1],[1,1,-1],[-1,1,-1],[-1,-1,1],[1,-1,1],[1,1,1],[-1,1,1]])*size
    Rx=np.array([[1,0,0],[0,np.cos(phi),-np.sin(phi)],[0,np.sin(phi),np.cos(phi)]]); Ry=np.array([[np.cos(theta),0,np.sin(theta)],[0,1,0],[-np.sin(theta),0,1]]); Rz=np.array([[np.cos(psi),-np.sin(psi),0],[np.sin(psi),np.cos(psi),0],[0,0,1]])
    rotated_points = (Rz@Ry@Rx @ points.T).T; projected_points = rotated_points[:, :2] + np.array(center_pos)
    edges = [[0,1],[1,2],[2,3],[3,0],[4,5],[5,6],[6,7],[7,4],[0,4],[1,5],[2,6],[3,7]]
    for edge in edges: pygame.draw.line(surface, color, projected_points[edge[0]], projected_points[edge[1]], width)

# --- Pygame 및 시뮬레이션 파라미터 ---
pygame.init()
WIDTH, HEIGHT = 1000, 800
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("3D 파티클 필터 시뮬레이션 (짐벌락 테스트)")
clock = pygame.time.Clock(); font = pygame.font.SysFont("Malgun Gothic", 30)

dt = 0.05; num_steps = 1000
NUM_PARTICLES = 2000 # 파티클 개수 (늘릴수록 정확해지지만 느려짐)
gyro_noise_std = 0.05; accel_noise_std = 0.1
Q = np.eye(3) * gyro_noise_std**2
R = np.eye(2) * accel_noise_std**2

# 단계 1: 데이터 생성 (이전과 동일)
true_states = []; gyro_inputs = []; accel_measurements = []
def get_gimbal_lock_gyro(t):
    if t < 5: return np.array([0, np.pi / 10.5, 0]) # Pitch를 90도로 만듦
    else: return np.array([0.5, 0, 0.5]) # 90도 상태에서 Roll, Yaw 시도
current_true_state = np.zeros(3)
for k in range(num_steps):
    t = k * dt; true_gyro = get_gimbal_lock_gyro(t)
    current_true_state = state_transition_function(current_true_state, true_gyro, dt)
    true_states.append(current_true_state.copy())
    gyro_inputs.append(true_gyro + np.random.normal(0, gyro_noise_std, 3))
    accel_measurements.append(measurement_function(current_true_state) + np.random.normal(0, accel_noise_std, 2))

# ==============================================================
# 단계 2: 파티클 필터 적용
# ==============================================================
print("단계 2: 파티클 필터 적용 중...")
x_est_initial = np.array([0.1, -0.1, 0.05])
P_initial = np.eye(3) * 0.2
pf = ParticleFilter(NUM_PARTICLES, x_est_initial, P_initial, Q, R, state_transition_function, measurement_function)
estimated_states = []
for k in range(num_steps):
    pf.predict(u=gyro_inputs[k], dt=dt)
    pf.update(z=accel_measurements[k])
    estimated_states.append(pf.estimate())
    pf.resample_if_needed()

# ==============================================================
# 단계 3: 시각화
# ==============================================================
print("단계 3: 시각화 시작...")
running = True; frame_index = 0
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT: running = False
    screen.fill((20, 20, 40))
    if frame_index < num_steps:
        true_angles = true_states[frame_index]
        est_angles = estimated_states[frame_index]
        draw_cube(screen, true_angles, (WIDTH*0.25, HEIGHT/2), 100, (100, 150, 255), 3)
        draw_cube(screen, est_angles, (WIDTH*0.75, HEIGHT/2), 100, (100, 255, 150), 3)
        
        # ... (텍스트 표시 부분은 이전과 동일) ...
        def draw_text(x, y, text, value, unit):
             label = font.render(f"{text}: {np.rad2deg(value):.1f}{unit}", True, (255, 255, 255))
             screen.blit(label, (x, y))
        draw_text(50, 50, "실제 Roll (φ)", true_angles[0], "°"); draw_text(50, 80, "실제 Pitch (θ)", true_angles[1], "°"); draw_text(50, 110, "실제 Yaw (ψ)", true_angles[2], "°")
        draw_text(WIDTH - 250, 50, "추정 Roll (φ)", est_angles[0], "°"); draw_text(WIDTH - 250, 80, "추정 Pitch (θ)", est_angles[1], "°"); draw_text(WIDTH - 250, 110, "추정 Yaw (ψ)", est_angles[2], "°")
        if abs(np.rad2deg(true_angles[1])) > 85:
            gimbal_text = pygame.font.SysFont("Malgun Gothic", 36, bold=True).render("짐벌락 구간!", True, (255, 50, 50))
            screen.blit(gimbal_text, gimbal_text.get_rect(center=(WIDTH/2, 100)))

        frame_index += 1
    else:
        text_done = pygame.font.SysFont("Malgun Gothic", 50, bold=True).render("시뮬레이션 완료", True, (0, 150, 0))
        screen.blit(text_done, text_done.get_rect(center=(WIDTH/2, HEIGHT/2)))
    pygame.display.flip()
    clock.tick(60)
pygame.quit()

단계 2: 파티클 필터 적용 중...
단계 3: 시각화 시작...
